In [1]:
! pip install flask

In [7]:
from watchdog.events import EVENT_TYPE_CREATED, EVENT_TYPE_MODIFIED

In [8]:
from flask import Flask, render_template, request, jsonify
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO
import base64
from datetime import datetime
from pathlib import Path

app = Flask(__name__)

class DataFrameAnalyzer:
    def __init__(self):
        self.df = None
        self.operations = ["Statistiques", "Valeur la plus fréquente", "Valeurs distinctes", "Valeurs manquantes", "Histogrammes", "Box Plot"]
        self.results_df = pd.DataFrame()
        self.file_name = None
        self.file_path = None

    def analyze(self, selected_columns):
        results_data = []

        for column in selected_columns:
            result_texts = []

            for operation in self.operations:
                if operation == "Valeur la plus fréquente":
                    result_texts.append(("Valeur la plus fréquente", self.df[column].mode().iloc[0]))
                elif operation == "Valeurs distinctes":
                    result_texts.append(("Valeurs distinctes", f"{', '.join(map(str, self.df[column].unique()))}"))
                elif operation == "Valeurs manquantes":
                    result_texts.append(("Missing", self.df[column].isnull().mean() * 100))
                elif operation == "Statistiques":
                    if pd.api.types.is_numeric_dtype(self.df[column]):
                        result_texts.append(("min", self.df[column].min()))
                        result_texts.append(("mean", self.df[column].mean()))
                        result_texts.append(("max", self.df[column].max()))
                        result_texts.append(("std", self.df[column].std()))
                    else:
                        result_texts.append((f"La colonne {column} n'est pas numérique, pas de stats possibles", None))
                elif operation == "Histogrammes":
                    image_path = self.create_image(column, chart="histogram")
                    result_texts.append(("histogram", image_path))
                elif operation == "Box Plot":
                    if pd.api.types.is_numeric_dtype(self.df[column]):
                        image_path = self.create_image(column, chart="box plot")
                        result_texts.append(("box plot", image_path))
                    else:
                        result_texts.append((f"La colonne {column} n'est pas numérique, box plot impossible", None))

            results_data.append({"Colonne": column, "Resultats": result_texts})

        self.results_df = pd.DataFrame(results_data)

    def create_image(self, column, chart):
        data = self.df[column]
        figure, ax = plt.subplots(figsize=(4, 3))

        if chart == "histogram":
            ax.hist(data, bins='auto')
            ax.set_title(f'Histogramme de {column}')
        elif chart == "box plot":
            ax.boxplot(data)
            ax.set_title(f'Box plot de {column}')

        image_path = self.save_image(figure, column, chart)
        plt.close(figure)
        return image_path

    def save_image(self, figure, column, chart, save_path='static/images'):
        if not Path(save_path).exists():
            Path(save_path).mkdir(parents=True)

        current_datetime = datetime.now().strftime("%Y%m%d_%H%M%S")
        file_name = f"{column}_{chart.replace(' ', '_')}_{current_datetime}.png"
        file_path = Path(save_path) / file_name
        figure.savefig(file_path)
        return f'/{file_path}'

    def load_file(self, file_path):
        if file_path:
            if file_path.endswith('.csv'):
                self.df = pd.read_csv(file_path)
            elif file_path.endswith('.xlsx'):
                self.df = pd.read_excel(file_path)
            else:
                return "Format de fichier non pris en charge."

            self.file_name = Path(file_path).name
            self.file_path = file_path
            return None
        return "Erreur lors du chargement du fichier."

data_frame_analyzer = DataFrameAnalyzer()

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/load_file', methods=['POST'])
def load_file():
    file_path = request.form.get('file_path')
    error = data_frame_analyzer.load_file(file_path)
    if error:
        return jsonify({"error": error})
    return jsonify({"success": True, "file_name": data_frame_analyzer.file_name})

@app.route('/analyze', methods=['POST'])
def analyze():
    selected_columns = request.form.getlist('selected_columns[]')
    data_frame_analyzer.analyze(selected_columns)
    return jsonify({"success": True, "results": data_frame_analyzer.results_df.to_dict(orient='records')})

if __name__ == '__main__':
    app.run(port=5001, debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5001
INFO:werkzeug:Press CTRL+C to quit


ImportError: cannot import name 'EVENT_TYPE_OPENED' from 'watchdog.events' (/Users/Roberto/opt/anaconda3/lib/python3.8/site-packages/watchdog/events.py)